In [6]:
import numpy as np

In [7]:
class DepthwiseConv2D:
    """Computes convolution given the input parameters"""
    def __init__(self, params, verbose=True, debug=False):
        super(DepthwiseConv2D, self).__init__()
        default_params = {
        'stride': 1,
        'dilation': 1,
        'padding': 0
        }
        for key, value in default_params.items():
            setattr(self, key, params.get(key, value))
        self.inp_n = params['inp_n']
        self.inp_c = params['inp_c']
        self.inp_h = params['inp_h']
        self.inp_w = params['inp_w']
        self.ker_c = params['ker_c']
        self.ker_h = params['ker_h']
        self.ker_w = params['ker_w']
        self.num_ker = params['num_ker']
        self.input_img = None
        self.kernels = None
        self.out_n = None
        self.out_c = None
        self.out_h = None
        self.out_w = None
        self.output = None
        self.verbose = verbose
        self.verboseprint = print if self.verbose else lambda *a, **k: None
        self.debug = debug
        self.debugprint = print if self.debug else lambda *a, **k: None
        self.print_params()
    
    def print_params(self):
        self.verboseprint('*** parameters ***')
        self.verboseprint('input batches: {}, input channels: {}, input height: {}, input weight: {}'.format(self.inp_n, self.inp_c, self.inp_h, self.inp_w))
        self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.num_ker, self.ker_c, self.ker_h, self.ker_w))
        self.verboseprint('stride: {}, dilation factor: {}, padding: {}'.format(self.stride, self.dilation, self.padding))
        self.verboseprint('\n')
        
    def create_input_img(self):
        # create image from the input parameters
        input_img = np.random.rand(self.inp_n, self.inp_c, self.inp_h, self.inp_w) # define a random image based on the input parameters
        if self.debug:
            for b in range(self.inp_n):
                input_img[b] = (b+1) * np.ones_like(input_img[b]) # define an image of all ones (twos etc.) based on the input parameters
        self.verboseprint('*** input image ***')
        self.verboseprint('input batches: {}, input channels: {}, input height: {}, input weight: {}'.format(self.inp_n, self.inp_c, self.inp_h, self.inp_w))
        self.verboseprint(input_img)
        self.verboseprint('\n')
        self.input_img = input_img
        self.add_padding()
        
    def add_padding(self):
        # add zero padding based on the input parameters
        if self.padding != 0:
            self.input_img = [[np.pad(channel,self.padding, 'constant', constant_values=0) for channel in batch] for batch in self.input_img]    
            self.inp_h += 2 * self.padding
            self.inp_w += 2 * self.padding
            self.verboseprint('*** padded input image ***')
            self.verboseprint('input batches: {}, input channels: {}, input height: {}, input weight: {}'.format(self.inp_n, self.inp_c, self.inp_h, self.inp_w))
            self.verboseprint(self.input_img)
            self.verboseprint('\n')
    
    def create_kernels(self):
        # create random kernels based on the input kernel parameters
        kernels = []
        self.verboseprint('*** kernels ***')
        self.verboseprint('kernels: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.num_ker, self.ker_c, self.ker_h, self.ker_w))
        for k in range(self.num_ker):
            kernel = np.random.rand(self.ker_c, self.ker_h, self.ker_w) # define a random kernel based on the kernel parameters
            if self.debug:
                kernel = k * np.ones_like(kernel)
            kernels.append(kernel)
            self.verboseprint('kernel {}'.format(k))
            self.verboseprint(kernel)
        self.verboseprint('\n')
        self.kernels = kernels
        self.dilate_kernels()
        
    def dilate_kernels(self):
        # dilate a kernel
        dil_ker_h = self.dilation * (self.ker_h - 1) + 1
        dil_ker_w = self.dilation * (self.ker_w - 1) + 1
        dil_kernels = []
        for kernel in self.kernels:
            dil_kernel = []
            for channel in kernel:
                dil_channel = np.zeros((dil_ker_h, dil_ker_w))
                for row in range(len(channel)):
                    for col in range(len(channel[0])):
                        dil_channel[self.dilation*row][self.dilation*col] = channel[row][col]
                dil_kernel.append(dil_channel.tolist())
            dil_kernels.append(dil_kernel)
        self.kernels, self.ker_h, self.ker_w = dil_kernels, dil_ker_h, dil_ker_w
        self.verboseprint('*** dilated kernels ***')
        self.verboseprint('kernels: {}, dilation factor: {}, kernel channels: {}, kernel height: {}, kernel weight: {}'.format(self.num_ker, self.dilation, self.ker_c, self.ker_h, self.ker_w))
        for k in range(self.num_ker):
            self.verboseprint('kernel {}'.format(k))
            self.verboseprint(self.kernels[k])
        self.verboseprint('\n')
        
    def compute_out_vol(self):
        # compute output volume from the input and kernel parameters
        out_n = int(self.inp_n)
        out_c = int(self.num_ker)
        out_h = int((self.inp_h - self.ker_h)/self.stride) + 1
        out_w = int((self.inp_w - self.ker_w)/self.stride) + 1
        self.out_n, self.out_c, self.out_h, self.out_w = out_n, out_c, out_h, out_w
        
    def convolve(self, h, w, ker_num, batch_num):
        # convolve kernel over the input slices
        self.debugprint('kernel indices, image indices')
        self.debugprint('[c, h, w]', '[n, c, h, w]')
        convol_sum = 0
        for h_ker in range(self.ker_h):
            for w_ker in range(self.ker_w):
                self.debugprint([ker_num, 0, h_ker, w_ker], [batch_num, ker_num, h_ker + self.stride*h, w_ker + self.stride*w])
                convol_sum += self.kernels[ker_num][0][h_ker][w_ker] * self.input_img[batch_num][ker_num][h_ker + self.stride*h][w_ker + self.stride*w]
        self.debugprint('\n')
        return convol_sum
    
    def create_output(self):
        # create output from the input and kernel parameters 
        self.compute_out_vol()
        self.output = np.zeros([self.out_n, self.out_c, self.out_h, self.out_w])
        # parse through every element of the output and compute the convolution value for that element
        for b in range(self.out_n):
            for k in range(self.num_ker):
                for h in range(self.out_h):
                    for w in range(self.out_w):
                        self.output[b, k, h, w] += self.convolve(h, w, k, b)
        self.verboseprint('*** output ***')
        output_shape = self.output.shape
        self.verboseprint('output batches: {}, ouput channels: {}, output height: {}, output weight: {}'.format(output_shape[0], output_shape[1], output_shape[2], output_shape[3]))
        assert((self.out_n, self.out_c, self.out_h, self.out_w) == output_shape)
        self.verboseprint(self.output)
        self.verboseprint('\n')

In [8]:
inp_n = 2 # number of input batches
inp_c, inp_h, inp_w = 2, 4, 4 # input channels, input height, input weight
ker_c, ker_h, ker_w = 1, 2, 2 # kernel channels (always equal to 1), kernel height, kernel weight
num_ker = inp_c # number of kernels (equal to number of input channels)
stride = 2 # stride (optional)
dilation = 1 # dilation factor (optional)
padding = 0 # padding (optional)
params = {'inp_n':inp_n, 'inp_c':inp_c, 'inp_h':inp_h, 'inp_w':inp_w, 'ker_c':ker_c, 'ker_h':ker_h, 
              'ker_w':ker_w, 'num_ker':num_ker, 'stride':stride, 'dilation':dilation, 'padding':padding}

dwconv2D = DepthwiseConv2D(params, debug=True)
dwconv2D.create_input_img()
dwconv2D.create_kernels()
dwconv2D.create_output()

*** parameters ***
input batches: 2, input channels: 2, input height: 4, input weight: 4
kernels: 2, kernel channels: 1, kernel height: 2, kernel weight: 2
stride: 2, dilation factor: 1, padding: 0


*** input image ***
input batches: 2, input channels: 2, input height: 4, input weight: 4
[[[[1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]]

  [[1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]
   [1. 1. 1. 1.]]]


 [[[2. 2. 2. 2.]
   [2. 2. 2. 2.]
   [2. 2. 2. 2.]
   [2. 2. 2. 2.]]

  [[2. 2. 2. 2.]
   [2. 2. 2. 2.]
   [2. 2. 2. 2.]
   [2. 2. 2. 2.]]]]


*** kernels ***
kernels: 2, kernel channels: 1, kernel height: 2, kernel weight: 2
kernel 0
[[[0. 0.]
  [0. 0.]]]
kernel 1
[[[1. 1.]
  [1. 1.]]]


*** dilated kernels ***
kernels: 2, dilation factor: 1, kernel channels: 1, kernel height: 2, kernel weight: 2
kernel 0
[[[0.0, 0.0], [0.0, 0.0]]]
kernel 1
[[[1.0, 1.0], [1.0, 1.0]]]


kernel indices, image indices
[c, h, w] [n, c, h, w]
[0, 0, 0, 0] [0, 0, 0, 0]
[0, 0, 0, 1] [

In [32]:
# input_img = np.random.rand(inp_c, inp_h, inp_w) # define a random image based on the input parameters
# input_img = np.ones_like(input_img) # define an image of all ones based on the input parameters
# print('input image')
# print(input_img)

In [33]:
# # method to dilate a kernel
# def dilate_kernels(dilation, kernels):
#     dil_ker_h = dilation * (ker_h - 1) + 1
#     dil_ker_w = dilation * (ker_w - 1) + 1
#     dil_kernels = []
#     for kernel in kernels:
#         dil_kernel = []
#         for channel in kernel:
#             dil_channel = np.zeros((dil_ker_h, dil_ker_w))
#             for row in range(len(channel)):
#                 for col in range(len(channel[0])):
#                     dil_channel[dilation*row][dilation*col] = channel[row][col]
#             dil_kernel.append(dil_channel.tolist())
#         dil_kernels.append(dil_kernel)
#     return dil_kernels, dil_ker_h, dil_ker_w

In [34]:
# # define random kernels based on the input kernel parameters
# kernels = []
# print('{} kernels'.format(num_ker))
# for k in range(num_ker):
#     kernel = np.random.rand(ker_c, ker_h, ker_w) # define a random kernel based on the kernel parameters
#     kernel = k * np.ones_like(kernel)
#     kernels.append(kernel)
#     print('kernel {}'.format(k))
#     print(kernel)
# print('kernel channels: {}, kernel height: {}, kernel weight: {}'.format(ker_c, ker_h, ker_w))

In [35]:
# # dilate kernels
# kernels, ker_h, ker_w = dilate_kernels(dilation, kernels)
# for k in range(num_ker):
#     print('dilated kernel {}'.format(k))
#     print(kernels[k])
# print('dilated kernel channels: {}, dilated kernel height: {}, dilated kernel weight: {}'.format(ker_c, ker_h, ker_w))

In [36]:
# # method to compute output volume from the input and kernel parameters
# def compute_out_vol(inp_c, inp_h, inp_w, ker_c, ker_h, ker_w, s, num_ker):
#     out_c = int(num_ker)
#     out_h = int((inp_h - ker_h)/s) + 1
#     out_w = int((inp_w - ker_h)/s) + 1
#     return out_c, out_h, out_w

In [37]:
# out_c, out_h, out_w = compute_out_vol(inp_c, inp_h, inp_w, ker_c, ker_h, ker_w, s, num_ker)
# print('ouput channels: {}, output height: {}, output weight: {}'.format(out_c, out_h, out_w))

In [38]:
# # method to convolve kernel over the input slices
# def convolve(c, h, w, ker_c, ker_h, ker_w, s, ker_num):
#     print('kernel indices, image indices')
#     print('[c, h, w]', '[c, h, w]')
#     convol_sum = 0
#     for c_ker in range(ker_c):
#         for h_ker in range(ker_h):
#             for w_ker in range(ker_w):
#                 print([c_ker, h_ker, w_ker], [c_ker, h_ker + s*h, w_ker + s*w])
# #                 convol_sum += kernels[ker_num][c_ker, h_ker, w_ker] * input_img[c_ker, h_ker + s*h, w_ker + s*w]
#                 convol_sum += kernels[ker_num][c_ker][h_ker][w_ker] * input_img[c_ker][h_ker + s*h][w_ker + s*w]
#     print('\n')
#     return convol_sum

In [39]:
# # method to create output from the input and kernel parameters 
# def create_output(inp_c, inp_h, inp_w, ker_c, ker_h, ker_w, s, num_ker):
#     out_c, out_h, out_w = compute_out_vol(inp_c, inp_h, inp_w, ker_c, ker_h, ker_w, s, num_ker)
#     output = np.zeros([out_c, out_h, out_w])
#     # parse through every element of the output and compute the convolution value for that element
#     for k in range(num_ker):
#         for h in range(out_h):
#             for w in range(out_w):
#                 for c in range(inp_c):
#                     output[k, h, w] += convolve(c, h, w, ker_c, ker_h, ker_w, s, k)
#     return output

In [40]:
# output = create_output(inp_c, inp_h, inp_w, ker_c, ker_h, ker_w, s, num_ker)
# print('output')
# output_shape = output.shape
# print('ouput channels: {}, output height: {}, output weight: {}'.format(output_shape[0], output_shape[1], output_shape[2]))
# assert((out_c, out_h, out_w) == output_shape)
# print(output)